1 - Importações

In [1]:
import pandas as pd
import time
import unicodedata
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerCareerStats

2 - Função de normalização

In [2]:
def normalize(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

3 - Dicionário com MVPs e temporadas

In [3]:
mvp_seasons = {
    "Kareem Abdul-Jabbar": ["1975-76", "1976-77", "1979-80"],
    "Bill Walton": ["1977-78"],
    "Moses Malone": ["1978-79", "1981-82", "1982-83"],
    "Larry Bird": ["1983-84", "1984-85", "1985-86"],
    "Magic Johnson": ["1986-87", "1988-89", "1989-90"],
    "Michael Jordan": ["1987-88", "1990-91", "1991-92", "1995-96", "1997-98"],
    "Charles Barkley": ["1992-93"],
    "Hakeem Olajuwon": ["1993-94"],
    "David Robinson": ["1994-95"],
    "Karl Malone": ["1996-97", "1998-99"],
    "Shaquille O'Neal": ["1999-00"],
    "Allen Iverson": ["2000-01"],
    "Tim Duncan": ["2001-02", "2002-03"],
    "Kevin Garnett": ["2003-04"],
    "Steve Nash": ["2004-05", "2005-06"],
    "Dirk Nowitzki": ["2006-07"],
    "Kobe Bryant": ["2007-08"],
    "LeBron James": ["2008-09", "2009-10", "2011-12", "2012-13"],
    "Derrick Rose": ["2010-11"],
    "Kevin Durant": ["2013-14"],
    "Stephen Curry": ["2014-15", "2015-16"],
    "Russell Westbrook": ["2016-17"],
    "James Harden": ["2017-18"],
    "Giannis Antetokounmpo": ["2018-19", "2019-20"],
    "Nikola Jokic": ["2020-21", "2021-22", "2023-24"],
    "Joel Embiid": ["2022-23"],
    "Shai Gilgeous-Alexander": ["2024-25"]
}

 4 - Obter IDs dos jogadores

In [4]:
player_dict = players.get_players()
mvp_ids = {}

for name in mvp_seasons.keys():
    found = [p for p in player_dict if normalize(p['full_name']) == normalize(name)]
    if found:
        mvp_ids[name] = found[0]['id']
    else:
        print(f" Não encontrei {name} na API")

5 - Coletar estatísticas das temporadas MVP

In [5]:
mvp_data = []
for name, seasons in mvp_seasons.items():
    print(f"Coletando dados de {name}...")
    pid = mvp_ids.get(name)

    if pid is None:
        print(f" ID não encontrado para {name}")
        continue

    career = PlayerCareerStats(player_id=pid)
    df = career.get_data_frames()[0]
    df['SEASON_ID'] = df['SEASON_ID'].astype(str)
    df_filtered = df[df['SEASON_ID'].isin(seasons)].copy()
    df_filtered["PLAYER_NAME"] = name

    if not df_filtered.empty:
        mvp_data.append(df_filtered)

    time.sleep(1)  # evitar bloqueio da API

Coletando dados de Kareem Abdul-Jabbar...
Coletando dados de Bill Walton...
Coletando dados de Moses Malone...
Coletando dados de Larry Bird...
Coletando dados de Magic Johnson...
Coletando dados de Michael Jordan...
Coletando dados de Charles Barkley...
Coletando dados de Hakeem Olajuwon...
Coletando dados de David Robinson...
Coletando dados de Karl Malone...
Coletando dados de Shaquille O'Neal...
Coletando dados de Allen Iverson...
Coletando dados de Tim Duncan...
Coletando dados de Kevin Garnett...
Coletando dados de Steve Nash...
Coletando dados de Dirk Nowitzki...
Coletando dados de Kobe Bryant...
Coletando dados de LeBron James...
Coletando dados de Derrick Rose...
Coletando dados de Kevin Durant...
Coletando dados de Stephen Curry...
Coletando dados de Russell Westbrook...
Coletando dados de James Harden...
Coletando dados de Giannis Antetokounmpo...
Coletando dados de Nikola Jokic...
Coletando dados de Joel Embiid...
Coletando dados de Shai Gilgeous-Alexander...


6 – Concatenar, organizar, ajustar, exibir e salvar DataFrame

In [6]:
if mvp_data:
    # Concatenar todos os DataFrames
    df_mvp = pd.concat(mvp_data, ignore_index=True)

    # Remover colunas desnecessárias
    df_mvp = df_mvp.drop(columns=["PLAYER_ID", "LEAGUE_ID", "TEAM_ID"], errors="ignore")

    # Converter colunas que devem ser inteiras, tratando NaNs
    int_columns = [
        "GP", "GS", "MIN", "FGM", "FGA", "FG3M", "FG3A", "FTM", "FTA",
        "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS"
    ]
    for col in int_columns:
        if col in df_mvp.columns:
            # Substituir NaN por 0 antes da conversão
            df_mvp[col] = df_mvp[col].fillna(0).astype(int)

    # Exibir os primeiros registros
    print("\n Primeiros registros do DataFrame MVP:")
    print(df_mvp.head())

    # Salvar em CSV
    df_mvp.to_csv("mvp_stats.csv", index=False)
else:
    df_mvp = pd.DataFrame()
    print(" Nenhum dado foi coletado.")


 Primeiros registros do DataFrame MVP:
  SEASON_ID TEAM_ABBREVIATION  PLAYER_AGE  GP  GS   MIN  FGM   FGA  FG_PCT  \
0   1975-76               LAL        29.0  82  82  3379  914  1728   0.529   
1   1976-77               LAL        30.0  82  82  3016  888  1533   0.579   
2   1979-80               LAL        33.0  82  81  3143  835  1383   0.604   
3   1977-78               POR        25.0  58   0  1929  460   882   0.522   
4   1978-79               HOU        24.0  82  82  3390  716  1325   0.540   

   FG3M  ...  OREB  DREB   REB  AST  STL  BLK  TOV   PF   PTS  \
0     0  ...   272  1111  1383  413  119  338    0  292  2275   
1     0  ...   266   824  1090  319  101  261    0  262  2152   
2     0  ...   190   696   886  371   81  280  297  216  2034   
3     0  ...   118   648   766  291   60  146  206  145  1097   
4     0  ...   587   857  1444  147   79  119  326  223  2031   

           PLAYER_NAME  
0  Kareem Abdul-Jabbar  
1  Kareem Abdul-Jabbar  
2  Kareem Abdul-Jabbar  
